In [1]:
import os
import networkx as nx
import numpy as np
import pandas as pd
import math
import pyflagser
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
G=nx.read_gml("Fully Observable/37busEx.gml")

In [3]:
A = nx.to_numpy_array(G)
print(A)

[[0. 1. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [4]:
N=list(G.nodes)
print(N)
print(len(N))
E=list(G.edges)
print(E)
print(len(E))

['sourcebus', '799', '709', '775', '701', '702', '705', '713', '703', '727', '730', '704', '714', '720', '742', '712', '706', '725', '707', '724', '722', '708', '733', '732', '731', '710', '735', '736', '711', '741', '740', '718', '744', '734', '737', '738', '728', '729', '799r']
39
[('sourcebus', '799'), ('799', '799r'), ('709', '775'), ('709', '731'), ('709', '708'), ('709', '730'), ('701', '702'), ('701', '799r'), ('702', '705'), ('702', '713'), ('702', '703'), ('705', '742'), ('705', '712'), ('713', '704'), ('703', '727'), ('703', '730'), ('727', '744'), ('704', '714'), ('704', '720'), ('714', '718'), ('720', '707'), ('720', '706'), ('706', '725'), ('707', '724'), ('707', '722'), ('708', '733'), ('708', '732'), ('733', '734'), ('710', '735'), ('710', '736'), ('710', '734'), ('711', '741'), ('711', '740'), ('711', '738'), ('744', '728'), ('744', '729'), ('734', '737'), ('737', '738')]
38


In [5]:
import pickle
with open('Fully Observable/fullanomalydataset.pkl', 'rb') as g:
    P0Data = pickle.load(g)

In [6]:
Class=[]
for i in range(len(P0Data)):
    Class.append(P0Data[i]["Anomaly Check"])
print(len(Class))
i=0
l=0
m=0
while i < len(Class):
    if Class[i] == 'No':
        l=l+1
        Class[i] = 0
    if Class[i] == 'Yes':
        m=m+1
        Class[i] = 1
    i += 1
#print(Class)
print(l)
print(m)

500
271
229


In [7]:
def Average(lst):
    return sum(lst) / len(lst)
AverageVoltage=[]
TimeSeries_Voltage=P0Data[330]["TimeSeries_Voltage"]
Voltage=TimeSeries_Voltage[3]
for y in Voltage:
    AverageVoltage.append(Average(list(y)))
AverageVoltage = [i * 10000-9999 for i in AverageVoltage]
print(AverageVoltage)

[0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211, 0.9410907454130211]


In [8]:
N_Senario=len(P0Data)
print(N_Senario)

500


In [9]:
def TimeSeries_Fe(TS_array, F_voltage):
    betti_0=[]
    for k in range(len(TS_array)):
        fec=[]
        AverageVoltage=[]
        Voltage=TS_array[k]
        for y in Voltage:
            AverageVoltage.append(Average(list(y)))
        AverageVoltage = [i * 10000-9999 for i in AverageVoltage]
        for p in range(len(F_voltage)):
            n_active = np.where(np.array(AverageVoltage) > F_voltage[p])[0].tolist()
            Active_node=np.unique(n_active)
            if (len(Active_node)==0):
                fec.append(0)
            else:
                b=A[Active_node,:][:,Active_node]
                my_flag=pyflagser.flagser_unweighted(b, min_dimension=0, max_dimension=2, directed=False, coeff=2, approximation=None)
                x = my_flag["betti"]
                fec.append(x[0])
            n_active.clear()
        betti_0.append(fec)
    return betti_0

In [ ]:
TimeSeries_Voltage=P0Data[330]["TimeSeries_Voltage"]

In [10]:
F_voltage=[0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.7]
betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)

In [11]:
print(betti)

[[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]


In [12]:
#F_voltage=[0.98,0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.7]
F_voltage=[0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.87,0.86,0.7]
Betti_0=[]
N_Senario=len(P0Data)
for i in range(N_Senario):
    print("\rProcessing file {} ({}%)".format(i, 100*i//(N_Senario-1)), end='', flush=True)
    TimeSeries_Voltage=P0Data[i]["TimeSeries_Voltage"]
    betti=TimeSeries_Fe(TimeSeries_Voltage, F_voltage)
    Betti_0.append(betti)

Processing file 499 (100%)

In [13]:
from sklearn.model_selection import KFold
# Define the K-fold Cross Validator
num_folds=10
kfold = KFold(n_splits=num_folds, shuffle=True)

In [15]:
X = Betti_0.copy()
y = Class.copy()
X=np.array(X)
y=np.array(y)

In [17]:
num_samples = 500
num_timesteps = 24
num_features = len(F_voltage)
num_classes = 2

In [18]:
loss_per_fold=[]
acc_per_fold=[]
fold_no = 1
for train, test in kfold.split(X, y):
    # Define the LSTM model
    model = Sequential([
        LSTM(10, activation='tanh',recurrent_activation='sigmoid',dropout=0.0,input_shape=(num_timesteps, num_features)),
        Dense(num_classes, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X[train], y[train], epochs=50, batch_size=32)
    # Evaluate the model
    #loss, accuracy = model.evaluate(X[test], y[test])
    #print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")
    scores = model.evaluate(X[test], y[test])
    
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

      # Increase fold number
    fold_no = fold_no + 1

Epoch 1/50


2024-03-03 22:41:41.943791: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-03 22:41:41.943843: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-03 22:41:41.943860: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-03 22:41:41.944183: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-03 22:41:41.944214: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-03 22:41:42.487640: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:41:42.567537: I t

15/15 [==============================] - 1s 11ms/step - loss: 0.6890 - accuracy: 0.5600
Epoch 2/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6765 - accuracy: 0.5689
Epoch 3/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6710 - accuracy: 0.5378
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6659 - accuracy: 0.5378
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6607 - accuracy: 0.5378
Epoch 6/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6545 - accuracy: 0.5467
Epoch 7/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6478 - accuracy: 0.5511
Epoch 8/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6390 - accuracy: 0.7311
Epoch 9/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6291 - accuracy: 0.8044
Epoch 10/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6057 - accuracy: 0.7089
Epoch 11/50
15/15 [==========

2024-03-03 22:41:48.883439: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:41:48.918521: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:41:49.396082: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:41:49.473979: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:41:49.534212: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 10ms/step - loss: 0.7101 - accuracy: 0.5444
Epoch 2/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6917 - accuracy: 0.5467
Epoch 3/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6758 - accuracy: 0.5467
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6675 - accuracy: 0.6333
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6548 - accuracy: 0.5578
Epoch 6/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6362 - accuracy: 0.5844
Epoch 7/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6108 - accuracy: 0.7156
Epoch 8/50
15/15 [==============================] - 0s 8ms/step - loss: 0.5725 - accuracy: 0.7689
Epoch 9/50
15/15 [==============================] - 0s 8ms/step - loss: 0.5334 - accuracy: 0.7978
Epoch 10/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4895 - accuracy: 0.8067
Epoch 11/50
15/15 [==========

...

2024-03-03 22:42:43.130969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:42:43.167586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Score for fold 9: loss of 0.36837127804756165; accuracy of 77.99999713897705%
Epoch 1/50


2024-03-03 22:42:43.667068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:42:43.744833: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:42:43.820454: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


15/15 [==============================] - 1s 14ms/step - loss: 0.6979 - accuracy: 0.5311
Epoch 2/50
15/15 [==============================] - 0s 9ms/step - loss: 0.6629 - accuracy: 0.5778
Epoch 3/50
15/15 [==============================] - 0s 8ms/step - loss: 0.6216 - accuracy: 0.8111
Epoch 4/50
15/15 [==============================] - 0s 8ms/step - loss: 0.5519 - accuracy: 0.8578
Epoch 5/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4850 - accuracy: 0.8533
Epoch 6/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4443 - accuracy: 0.8556
Epoch 7/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4085 - accuracy: 0.8578
Epoch 8/50
15/15 [==============================] - 0s 8ms/step - loss: 0.3865 - accuracy: 0.8578
Epoch 9/50
15/15 [==============================] - 0s 9ms/step - loss: 0.3794 - accuracy: 0.8578
Epoch 10/50
15/15 [==============================] - 0s 8ms/step - loss: 0.3916 - accuracy: 0.8556
Epoch 11/50
15/15 [==========

2024-03-03 22:42:50.223215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-03-03 22:42:50.260763: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [19]:
print(acc_per_fold)

[83.99999737739563, 86.00000143051147, 86.00000143051147, 92.00000166893005, 89.99999761581421, 89.99999761581421, 86.00000143051147, 80.0000011920929, 77.99999713897705, 86.00000143051147]


In [21]:
import statistics
mean_value = statistics.mean(acc_per_fold)
std_dev = statistics.stdev(acc_per_fold)
print("Mean using LSTM", mean_value)
print("Standard Deviation:", std_dev)

Mean using LSTM 85.799999833107
Standard Deviation: 4.366539733197398
